#  Algoritmo para el análisis tematico de documentos digitales

#### Luis Roberto Polo Bautista
#### Karen Vanessa Martínez Acevedo 

#### El objetivo del algoritmo es la modelación de temas de documentos digitales en español e inglés, que sirva como herramienta de apoyo al análisis temático dentro de la organización de la información.

### Extracción del texto completo de documentos en formato PDF

#### En esta parte se utilizó el módulo PyMuPDF de Python, el cual, entre otras funciones permite la identificación y extracción del texto de diferentes tipos de documentos exportándolos a archivos editables. En función al objetivo del articulo, se implementó este módulo para importar documentos en formato pdf, extraer el texto de este y exportándolo en un archivo de texto simple (archivo con extensión txt). A continuación, se muestra el pseudo-código que se utilizó en este proceso. 

In [ ]:
# importar los módulos necesario
import fitz

In [ ]:
# Importar el documento de texto en formato PDF
documento_pdf="c:/"#<-----Escribe la ruta donde se encuentra el documento PDF
# Leer el documento mediante el módulo PyMuPDF
documento=fitz.open(documento_pdf)
# Mostrar el número de páginas y los datos bibliográficos
print ("Número de páginas del documento: ",documento.pageCount)
print ("Datos bibligráficos: ",documento.metadata)

In [ ]:
# Declarar un variable que contenga la extensión del nuevo archivo en formato txt
documento_txt=open(documento_pdf + ".txt", "wb")
 
for paginas in documento: # Para todas las páginas del documento  hacer: 
    texto=paginas.getText().encode("utf8") # Extracción del texto en formato UTF-8
    documento_txt.write(texto) # Exportar el texto en un archivo con la extensión definida
    documento_txt.write(b"\n-----\n")# Definir dentro del archivo las divisiones de cada página
documento_txt.close() #Cerrar el documento

### Modelación de tópicos relevantes visualizándolos en HTML

#### En la sección de identificación de tópicos se utilizaron los módulos numpy y sklearn para el cálculo de frecuencia y vectores de palabras y el modelo de Asignación Latente de Dirichlet (ALD) para la identificación de los tópicos relevantes. Para la visualización dinámica de los temas por medio de html se utilizó el módulo pyLDAvis. 

In [ ]:
# Importar los módulos necesarios
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
from sklearn.decomposition import LatentDirichletAllocation as LDA
import os
from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer 

os.chdir('..')

In [ ]:
# Importar el archivo de texto simple con el preprocesamiento realizado 
with open ('c:/.pdf.txt', 'r', encoding="utf-8") as archivo: #<-----Escribe la ruta donde se encuentra el documento txt
    texto= archivo.read()

In [ ]:
# Eliminar los artículos, pronombres, preposiciones, etc., del texto 
agregar = ['No', 'text', 'returned', 'unable', 'to', 'reach', 'website.', 'one', 'new', '--', 'us'] #<---Escribe las palabras que no quiere que sean consideradas a parte de las palabras vacías
stop_words= (stopwords.words(spanish)) # para un texto en inglés, reemplazar 'spanish' por 'english'
stop_words.extend (agregar)
word_tokens= word_tokenize(texto.lower())
word_tokens= list(filter(lambda token: token not in string.punctuation, word_tokens))

In [ ]:
filtro=[]
for palabra in word_tokens: 
    if palabra not in stop_words:
        filtro.append(palabra)

In [ ]:
# normalizar todas las formas de una misma palabra 
lemmatizer = WordNetLemmatizer()
lemmatized_output = [lemmatizer.lemmatize(w) for w in filtro]

In [ ]:
# Calcular los vectores de las palabras individuales que conforman cada frase 
vectores_palabras = CountVectorizer()
transformar_vectores = vectores_palabras.fit_transform(lemmatized_output)

In [ ]:
# Crear una función que extraiga los vectores de las palabras y los guarde en una variable
def mostrar_tópicos(model, vectores_palabras, n_top_words):
    palabras = vectores_palabras.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([palabras[i]
                        for i in topic.argsort()[:-n_top_words - 1:- 1]]))

In [ ]:
# Declarar en una variable el número de tópicos a procesas
número_tópicos = 4
# Declarar en una variable el número de palabras relacionadas a los tópicos 
número_palabras = 30

In [ ]:
# Procesar los datos ingresados con base a modulo LDA
lda = LDA(n_components=número_tópicos, n_jobs=-1)
lda.fit(transformar_vectores)

In [ ]:
# Mostrar los datos procesados con base al vector asignado a cada palabra, así como el número de palabras relacionados a los tópicos 
print("Tópicos encontrados mediante LDA:")
mostrar_tópicos(lda, vectores_palabras, número_palabras)

In [ ]:
# Declarar en una variable el nombre del archivo que mostrara los temas de forma dinámica 
LDAvis_data_filepath = os.path.join('./tópic_'+str(número_tópicos))#<---- Escribe el nomnbre del archivo
# Procesar los datos para que se puedan mostrar de forma dinámica
if 1 == 1:LDAvis_prepared = sklearn_lda.prepare(lda, transformar_vectores, vectores_palabras)

In [ ]:
# Preparar el archivo con los datos a mostrar
with open(LDAvis_data_filepath, 'wb') as f:
    pickle.dump(LDAvis_prepared, f)
        
with open(LDAvis_data_filepath, 'rb',) as f:
    LDAvis_prepared = pickle.load(f)

In [ ]:
# Guardar el archivo en una ruta determinada con extensión HTML

pyLDAvis.save_html(LDAvis_prepared, 'c:/'+ str(LDAvis_data_filepath) +'.html')#<-----Escribe la ruta donde se quiere guardar el archivo HTML